In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
unnormed_weights = inputs @ inputs.T
unnormed_weights.shape

torch.Size([6, 6])

In [4]:
import torch.nn.functional as F

normed_weights = F.softmax(unnormed_weights, dim=-1)
normed_weights.sum(axis=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [5]:
context_vectors = normed_weights @ inputs
context_vectors.shape

torch.Size([6, 3])

In [6]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):  # x.shape = (batch_size, seq_len, d_in)
        queries = self.w_query(x) # queries.shape = (batch_size, seq_len, d_out)
        keys = self.w_key(x) # keys.shape = (batch_size, seq_len, d_out)
        values = self.w_value(x) # values.shape = (batch_size, seq_len, d_out)

        attn_scores = queries @ keys.T # attn_scores.shape = (batch_size, seq_len, seq_len)
        attn_weights = F.softmax(attn_scores / keys.shape[1]**0.5, dim=-1) # attn_weights.shape = (batch_size, seq_len, seq_len)

        context_vectors = attn_weights @ values # context_vectors.shape = (batch_size, seq_len, d_out)
        return context_vectors

In [7]:
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()

        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(p=dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        queries = self.w_query(x)
        keys = self.w_key(x)
        values = self.w_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(self.mask == 1, float('-inf'))
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        context_vectors = attn_weights @ values

        return context_vectors

In [8]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [9]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in=3, d_out=2, context_length=context_length, dropout=0.5)
context_vectors = ca(batch)
print(context_vectors.shape)

torch.Size([2, 6, 2])


In [10]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in=d_in, 
                             d_out=d_out, 
                             context_length=context_length, 
                             dropout=dropout, 
                             qkv_bias=qkv_bias) 
                             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [11]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3,2

mha = MultiHeadAttentionWrapper(
    d_in=d_in, 
    d_out=d_out, 
    context_length=context_length, 
    dropout=0.0, 
    num_heads=2
)

context_vectors = mha(batch)
print(context_vectors.shape)
print(context_vectors)

torch.Size([2, 6, 4])
tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


In [12]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, 'd_out must be divisible by num_heads'

        self.d_out = d_out
        self.d_head = d_out // num_heads
        self.num_heads = num_heads
        
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
        self.dropout = nn.Dropout(p=dropout)
        self.out_proj = nn.Linear(d_out, d_out)

    def forward(self, x):
        batch_size, seq_len, d_in = x.shape

        queries = self.w_query(x)
        keys = self.w_key(x)
        values = self.w_value(x)

        queries = queries.view(batch_size, seq_len, self.num_heads, self.d_head)
        values = values.view(batch_size, seq_len, self.num_heads, self.d_head)
        keys = keys.view(batch_size, seq_len, self.num_heads, self.d_head)

        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        keys = keys.transpose(1, 2)

        attn_scores = queries @ keys.transpose(-2, -1)
        attn_scores = attn_scores.masked_fill_(self.mask == 1, float('-inf'))
        attn_weights = torch.softmax(attn_scores / d_out**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        context_vectors = attn_weights @ values
        context_vectors = context_vectors.transpose(1, 2).contiguous()
        context_vectors = context_vectors.view(batch_size, seq_len, d_out)
        context_vectors = self.out_proj(context_vectors)
        return context_vectors

In [13]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(
    d_in=d_in, 
    d_out=d_out, 
    context_length=context_length, 
    dropout=0.0, 
    num_heads=2
)
context_vectors = mha(batch)
print(context_vectors.shape)
print(context_vectors)

torch.Size([2, 6, 2])
tensor([[[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]],

        [[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]]], grad_fn=<ViewBackward0>)


In [ ]:
from torch.utils.data import DataLoader, Dataset
import tiktoken
import torch
import torch.nn as nn

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            self.input_ids.append(token_ids[i:i+max_length])
            self.target_ids.append(token_ids[i+1:i+1+max_length])

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

# open 'the-verdict.txt' and read content
with open('the-verdict.txt', 'r') as file:
    txt = file.read()

vocab_size = 50257
output_dim = 256
max_len = 1024
context_length = max_len

token_embedding_layer = nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

max_length = 4
dataloader = create_dataloader(txt, batch_size=8, max_length=max_length, stride=max_length)

for x, y in dataloader:
    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings
